#**Optuna ia s Hyperparameter Tuning Freamework**

Link of Video description about Optuna: https://www.youtube.com/watch?v=E2b3SKMw934&t=0s

**Hyperparameter Tuning Technique**

1. GeridSearchCV (Describe in the video)

2. RandomSearchCV (Describe in the video)

3. BayesianSearch_Optuna (Describe in the video)

**5 Parameters of Optuna Freamework**

**`1. Study`**

**`2. Trial`**

**`3. Trial Parameter`**

**`4. Objective Function`**

**`5. Sampler (TPE Calculation)`**

**Others Features of Optuna**

1. Distributed Computing (For several server or device use)

2. Intrigrated Computing (sklearn, keras, pytorch, lmflow)


In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.6 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-01-23 08:19:21,955] A new study created in memory with name: no-name-5397e2b3-0e48-4994-81a6-c5e1ec56d228
[I 2025-01-23 08:19:23,672] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 167, 'max_depth': 20}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-01-23 08:19:26,111] Trial 1 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 172, 'max_depth': 10}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-01-23 08:19:29,354] Trial 2 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 148, 'max_depth': 4}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-01-23 08:19:31,524] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 66, 'max_depth': 9}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-01-23 08:19:33,237] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 65, 'max_depth': 8}. Best is trial 0 with value: 0.774674115

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 115, 'max_depth': 15}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-01-23 08:20:27,393] A new study created in memory with name: no-name-2663ced1-843e-47f1-8c5b-73fd7f67f2ac
[I 2025-01-23 08:20:27,983] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 110, 'max_depth': 4}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-01-23 08:20:28,920] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 167, 'max_depth': 20}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-01-23 08:20:29,259] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 57, 'max_depth': 17}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-01-23 08:20:29,649] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 68, 'max_depth': 20}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-01-23 08:20:30,362] Trial 4 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 148, 'max_depth': 3}. Best is trial 1 with value: 0.77467411

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 125, 'max_depth': 20}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-01-23 08:21:20,667] A new study created in memory with name: no-name-741c338c-94ed-416b-98a5-3b3da997d75f
[I 2025-01-23 08:21:21,344] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-23 08:21:22,331] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-01-23 08:21:22,817] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-01-23 08:21:23,631] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-01-23 08:21:24,238] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-01-23 08:21:39,614] A new study created in memory with name: no-name-793d4dbc-481f-407f-a28a-a11891d792e0
[I 2025-01-23 08:21:39,663] Trial 0 finished with value: 0.696461824953445 and parameters: {'classifier': 'SVM', 'C': 9.258681665717848, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.696461824953445.
[I 2025-01-23 08:21:41,743] Trial 1 finished with value: 0.7486033519553073 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 83, 'learning_rate': 0.2634345048210357, 'max_depth': 15, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.7486033519553073.
[I 2025-01-23 08:21:44,064] Trial 2 finished with value: 0.7392923649906891 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 111, 'learning_rate': 0.28582961967717035, 'max_depth': 12, 'min_samples_split': 3, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.7486033519553073.
[I 2025-01-23 08:21:45,895] Trial 3 finished with value: 0.75418994

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12381929984959765, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.696462,2025-01-23 08:21:39.620247,2025-01-23 08:21:39.663555,0 days 00:00:00.043308,9.258682,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.748603,2025-01-23 08:21:39.666311,2025-01-23 08:21:41.743254,0 days 00:00:02.076943,NaN,NaN,GradientBoosting,NaN,NaN,0.263435,15.0,4.0,7.0,83.0,COMPLETE
2,2,0.739292,2025-01-23 08:21:41.748220,2025-01-23 08:21:44.063544,0 days 00:00:02.315324,NaN,NaN,GradientBoosting,NaN,NaN,0.285830,12.0,5.0,3.0,111.0,COMPLETE
3,3,0.754190,2025-01-23 08:21:44.067507,2025-01-23 08:21:45.895356,0 days 00:00:01.827849,NaN,True,RandomForest,NaN,NaN,NaN,19.0,2.0,9.0,126.0,COMPLETE
4,4,0.759777,2025-01-23 08:21:45.905033,2025-01-23 08:21:48.861283,0 days 00:00:02.956250,NaN,True,RandomForest,NaN,NaN,NaN,4.0,7.0,6.0,265.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.728119,2025-01-23 08:22:27.299751,2025-01-23 08:22:27.402244,0 days 00:00:00.102493,0.329550,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.789572,2025-01-23 08:22:27.412857,2025-01-23 08:22:27.511305,0 days 00:00:00.098448,0.149386,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.787709,2025-01-23 08:22:27.515613,2025-01-23 08:22:27.576497,0 days 00:00:00.060884,0.180311,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-01-23 08:22:27.587626,2025-01-23 08:22:27.666499,0 days 00:00:00.078873,0.234275,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
GradientBoosting,11
RandomForest,11


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.742340
RandomForest,0.765194
SVM,0.775987


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [34]:
pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.9 MB/s eta 0:00:00


In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-01-23 08:25:18,557] A new study created in memory with name: no-name-baac70b8-6da3-43d9-a68c-731c5e4c2753


[0]	train-mlogloss:0.93268	eval-mlogloss:0.91790
[1]	train-mlogloss:0.73236	eval-mlogloss:0.70284
[2]	train-mlogloss:0.58953	eval-mlogloss:0.55111
[3]	train-mlogloss:0.47904	eval-mlogloss:0.43791
[4]	train-mlogloss:0.40131	eval-mlogloss:0.35233
[5]	train-mlogloss:0.34269	eval-mlogloss:0.28865
[6]	train-mlogloss:0.30557	eval-mlogloss:0.24989
[7]	train-mlogloss:0.27659	eval-mlogloss:0.21908
[8]	train-mlogloss:0.25260	eval-mlogloss:0.19254
[9]	train-mlogloss:0.24403	eval-mlogloss:0.18224
[10]	train-mlogloss:0.23784	eval-mlogloss:0.17317
[11]	train-mlogloss:0.23182	eval-mlogloss:0.16359
[12]	train-mlogloss:0.23093	eval-mlogloss:0.16077
[13]	train-mlogloss:0.23056	eval-mlogloss:0.15950
[14]	train-mlogloss:0.22995	eval-mlogloss:0.15914
[15]	train-mlogloss:0.22936	eval-mlogloss:0.15847
[16]	train-mlogloss:0.22882	eval-mlogloss:0.15609
[17]	train-mlogloss:0.22751	eval-mlogloss:0.15568
[18]	train-mlogloss:0.22579	eval-mlogloss:0.15716
[19]	train-mlogloss:0.22549	eval-mlogloss:0.15644
[20]	train

[I 2025-01-23 08:25:24,097] Trial 0 finished with value: 1.0 and parameters: {'lambda': 3.1284548541234395e-06, 'alpha': 1.2911272821511097e-07, 'eta': 0.21478165761469858, 'gamma': 0.0003026667910229203, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.7362907305498647, 'colsample_bytree': 0.6110384707206015}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.77014	eval-mlogloss:0.74138
[1]	train-mlogloss:0.57131	eval-mlogloss:0.52593
[2]	train-mlogloss:0.43836	eval-mlogloss:0.38536
[3]	train-mlogloss:0.35420	eval-mlogloss:0.29753
[4]	train-mlogloss:0.30406	eval-mlogloss:0.24554
[5]	train-mlogloss:0.26784	eval-mlogloss:0.20575
[6]	train-mlogloss:0.23846	eval-mlogloss:0.17331
[7]	train-mlogloss:0.23852	eval-mlogloss:0.17324
[8]	train-mlogloss:0.23779	eval-mlogloss:0.17304
[9]	train-mlogloss:0.22840	eval-mlogloss:0.15882
[10]	train-mlogloss:0.22815	eval-mlogloss:0.15938
[11]	train-mlogloss:0.22790	eval-mlogloss:0.15974
[12]	train-mlogloss:0.22702	eval-mlogloss:0.15942
[13]	train-mlogloss:0.22680	eval-mlogloss:0.15899
[14]	train-mlogloss:0.22660	eval-mlogloss:0.15950
[15]	train-mlogloss:0.22596	eval-mlogloss:0.15927
[16]	train-mlogloss:0.22600	eval-mlogloss:0.15919
[17]	train-mlogloss:0.22540	eval-mlogloss:0.15900
[18]	train-mlogloss:0.22547	eval-mlogloss:0.15888
[19]	train-mlogloss:0.22521	eval-mlogloss:0.15862
[20]	train

[I 2025-01-23 08:25:32,337] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.010097143391380403, 'alpha': 2.4844902058011584e-07, 'eta': 0.2844907080524637, 'gamma': 3.3290895952659024e-06, 'max_depth': 7, 'min_child_weight': 10, 'subsample': 0.8241221166361099, 'colsample_bytree': 0.8366493052882509}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.02967	eval-mlogloss:1.03633


[I 2025-01-23 08:25:59,862] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:1.03859	eval-mlogloss:1.03549
[1]	train-mlogloss:0.98254	eval-mlogloss:0.97305
[2]	train-mlogloss:0.93538	eval-mlogloss:0.92543
[3]	train-mlogloss:0.87302	eval-mlogloss:0.85850


[I 2025-01-23 08:26:00,018] Trial 49 pruned. Trial was pruned at iteration 4.


Best trial: {'lambda': 3.1284548541234395e-06, 'alpha': 1.2911272821511097e-07, 'eta': 0.21478165761469858, 'gamma': 0.0003026667910229203, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.7362907305498647, 'colsample_bytree': 0.6110384707206015}
Best accuracy: 1.0


In [36]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()